In [158]:
!pip install chromedriver_autoinstaller

In [1]:
!pip3 install beautifulsoup4
!pip install selenium
!pip install newspaper3k
!pip3 install keybert
nltk.download('all')

In [1]:
import requests 
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import time
import numpy
import random
import pickle
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime
import pandas as pd
from newspaper import Article
import os
import nltk

In [5]:
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
import subprocess
import shutil

try:
    shutil.rmtree(r"c:\chrometemp")  #쿠키 / 캐쉬파일 삭제
except FileNotFoundError:
    pass

subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"') # 디버거 크롬 구동


option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
try:
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
driver.implicitly_wait(10)

PermissionError: [WinError 32] 다른 프로세스가 파일을 사용 중이기 때문에 프로세스가 액세스 할 수 없습니다: 'c:\\chrometemp\\Default\\Affiliation Database'

In [2]:
from_date = '20230214'
to_date = '20230214'
searching_word = 'economy'
cwd = os.getcwd()
main_path = cwd + '/' + searching_word

In [4]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

createFolder(main_path)
createFolder(main_path + '/' + 'url')
createFolder(main_path + '/' + 'news_backup')
createFolder(main_path + '/' + 'news')
createFolder(main_path + '/' + 'keyword')

# 1 키워드 검색 뉴스 url 수집
def page2url(searching_word,date):
    searching_word = '"{}"'.format(searching_word)
    urls = []
    for start in range(0, 360, 10):
        main_url = 'https://www.google.co.kr/search?q={}&tbs=cdr:1,cd_min:{},cd_max:{}&tbm=nws&ei=dPP-Yu_eCJLL-Qb55bvQDA&start={}&sa=N&ved=2ahUKEwjv6Lvy69H5AhWSZd4KHfnyDso4ChDy0wN6BAgBEDk&biw=1536&bih=754&dpr=1.25'.format(searching_word,date,date,start)
        driver.get(url=main_url)
        elements = driver.find_elements(By.TAG_NAME, 'a')
        lnks = []
        for lnk in elements:
            lnk = str(lnk.get_attribute('href'))
            if 'google' not in lnk and lnk != 'None':
                lnks.append(lnk)
        if len(lnks) == 0:
            print(date, ' Url 수집 완료')
            break
        urls.extend(lnks)
        rand_value =random.uniform(4, 10)
        time.sleep(rand_value)
        
    return urls

datelist = pd.date_range(start=from_date, end=to_date).tolist()
dtlst = []
for d_t in datelist:
    d_t = str(d_t)[0:-9]
    d = datetime.strptime(d_t, '%Y%m%d')
    d = d.strftime('%m/%d/%Y')
    d = d[0].replace('0','') + d[1:]
    d = d[:-7] + d[-7].replace('0','') + d[-6:]
    dtlst.append(d)
    
    
Urls = dict()
for date in tqdm(dtlst):
    print(date,'Url 수집 시작')
    urls = page2url(searching_word,date)
    Urls[date] = urls
    
name = "Urls{}-{}.pickle".format(from_date,to_date)
name = name.replace('/','.')

with open(main_path + '/' + 'url/' + name,'wb') as f:
    pickle.dump(Urls,f)


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]


2/14/2023 Url 수집 시작


NameError: name 'driver' is not defined

In [49]:
name = "Urls{}-{}.pickle".format(from_date,to_date)
name = name.replace('/','.')
# 2  수집된 url로 news 크롤링 
with open(main_path + '/' + 'url/' + name, 'rb') as f:
    data = pickle.load(f)

# start = 0
for date in list(data):
    print(date, '크롤링 시작')
    cnt = 0
    tmp = []
    urls = []
    for idx,url in tqdm(enumerate(data[date])):
        try:
            article = Article(url)
            article.download()
            article.parse()
            v = article.text
            tmp.append(v)
            urls.append(url)
            cnt += 1
        except:
            pass
            tmp.append("null")
            urls.append(url)
            print(url)
    print("총 {}건 중 {} 건 크롤링 성공".format(idx+1,cnt))
    data[date] = [tmp,[url]]
    date = date.replace('/','-')
    with open(main_path + '/' + 'news_backup/' + 'news-' + date + '.pickle','wb') as f:
        pickle.dump(data,f)
    print(date,' 크롤링 완료')


2/14/2023 크롤링 시작


7it [00:10,  1.51s/it]

https://simpleflying.com/klm-economy-berlin-amsterdam-review/


15it [00:17,  1.01s/it]

https://www.mmh.com/article/enabling_the_circular_economy


...

https://www.miragenews.com/tasmanian-economy-thrives-in-nab-survey-947667/


226it [05:44,  2.95s/it]

https://www.dailynews.lk/2023/02/15/features/297376/development-earnest


230it [05:48,  1.51s/it]

총 230건 중 207 건 크롤링 성공
2-14-2023  크롤링 완료


In [61]:
 pd.DataFrame(data).T

,0
일자,20230214
news,[Create a free account and access your persona...
url,[https://www.iworldtoday.com/news/articleView....
kwd,"[[economic, international, world, article, ana..."


In [51]:
data = pd.DataFrame(data).T

data = data.rename({0:"news",1:"url"},axis= 1)

name = "News{}-{}.pickle".format(from_date,to_date)
name = name.replace('/','.')

with open(main_path + '/' + 'news/' + name,'wb') as f:
    pickle.dump(data,f)

In [52]:
from keybert import KeyBERT
import pickle
import re
from tqdm import tqdm
import pandas as pd
import os

In [53]:
def doc2key(cleaned_content):
    
    kw_model = KeyBERT(model='all-MiniLM-L6-v2')
    keywords = kw_model.extract_keywords(doc)
    
    n2_kwd = kw_model.extract_keywords(cleaned_content, keyphrase_ngram_range=(2, 2), stop_words='english',
                                  use_mmr=True, diversity=0.7, top_n=5)

    n1_kwd = kw_model.extract_keywords(cleaned_content, keyphrase_ngram_range=(1, 1), stop_words='english',
                                  use_mmr=True, diversity=0.7, top_n=45)
    for idx,i in enumerate(n2_kwd):
        n2_kwd[idx] = i[0]
    for idx,i in enumerate(n1_kwd):
        n1_kwd[idx] = i[0]  

    n1_kwd.extend(n2_kwd)
    kwd = n1_kwd
    return kwd

In [54]:
def makeKeywordLsit(data,nn_jj = False):
    kwd_list = []
    for docs in tqdm(data['news']):
        kwds = []
        for doc in tqdm(docs):
            try:
                cleaned_content = re.sub(r'[^\.\?\!\w\d\s]','',doc) # 문장단위로 끊기
                cleaned_content = cleaned_content.replace('\n',' ')
                cleaned_content = cleaned_content.lower()
                kwd = doc2key(cleaned_content)
                if nn_jj == True:
                    tokens_pos = nltk.pos_tag(kwd)
                    kwd_nn_jj = []
                    for word, pos in tokens_pos:
                        if 'NN' in pos or 'JJ' in pos:
                            kwd_nn_jj.append(word)
                    kwds.append(kwd_nn_jj)
                else:
                    kwds.append(kwd)
            except:
                print(doc)
        kwd_list.append(kwds)
    return kwd_list

In [55]:
name = "News{}-{}.pickle".format(from_date,to_date)
name = name.replace('/','.')

with open(main_path + '/' + 'news/' + name, 'rb') as f:
    data = pickle.load(f)

In [56]:
data = data.reset_index()
data = data.rename({"index":"date"},axis = 1)

idx = []
for x in data['date']:
    tmp = x.split('/')
    if len(tmp[0]) == 1:
        tmp[0] = '0'+tmp[0]
    if len(tmp[1]) == 1:
        tmp[1] = '0'+tmp[1]
    tmp = tmp[2] + tmp[0] + tmp[1]
    idx.append(''.join(tmp))

In [57]:
data['date'] = idx
data = data.rename({'date':'일자'},axis = 1)

In [62]:
data

,일자,news,url,kwd
0,20230214,[Create a free account and access your persona...,[https://www.iworldtoday.com/news/articleView....,"[[economic, international, world, article, ana..."


In [59]:
kwd_list = makeKeywordLsit(data, nn_jj = True)

 28%|██████████████████████▌                                                          | 64/230 [02:34<07:11,  2.60s/it]


 57%|█████████████████████████████████████████████▏                                  | 130/230 [05:06<03:11,  1.91s/it]


 85%|████████████████████████████████████████████████████████████████████▏           | 196/230 [07:36<01:11,  2.10s/it]


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [09:12<00:00, 552.36s/it]


In [63]:
data['kwd'] = kwd_list

In [64]:
name = "Keyword{}-{}.pickle".format(from_date,to_date)
name = name.replace('/','.')
with open(main_path + '/' + 'keyword/' + name,'wb') as f:
    pickle.dump(data,f)